In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Rio Grande Do Sul - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_RS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH,Rio Grande Do Sul - Consumo de Cimento (t)
0,2003-1,8.192273,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,135.543840,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,0.399576,0.775641,188.093
1,2003-2,8.186359,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,134.468148,2.068579e+08,8.912326e+06,18.211286,1.961183e+08,0.398811,0.775686,170.973
2,2003-3,8.180446,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,134.242963,2.070864e+08,8.917522e+06,18.215621,1.961742e+08,0.397898,0.775731,174.880
3,2003-4,8.174533,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,133.169391,2.073148e+08,8.922718e+06,18.219955,1.962300e+08,0.397042,0.775775,178.111
4,2003-5,8.168619,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,132.046874,2.075433e+08,8.927915e+06,18.224290,1.962858e+08,0.396050,0.775820,184.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.034199,NaN,NaN,NaN,NaN,0.832392,NaN,288.217
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.001080,NaN,NaN,NaN,NaN,0.829742,NaN,272.178
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.052788,NaN,NaN,NaN,NaN,0.826925,NaN,272.665
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.665535,NaN,NaN,NaN,NaN,0.823550,NaN,275.972


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
0,1.274013,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.577787,-1.677139,-1.973850,-2.280022,-2.069914,-1.362004,-1.691101
1,1.259068,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.629575,-1.660252,-1.934055,-2.253080,-2.045065,-1.371141,-1.675868
2,1.244123,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.640416,-1.643365,-1.894260,-2.226138,-2.020217,-1.382062,-1.660635
3,1.229178,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.692102,-1.626477,-1.854466,-2.199196,-1.995368,-1.392283,-1.645402
4,1.214233,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.746144,-1.609590,-1.814671,-2.172254,-1.970520,-1.404146,-1.630170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.966721,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.420132,1.083219,-0.768157,0.374323,0.568222,1.591021,0.807464
188,-0.973649,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.417824,1.070530,-0.760321,0.347418,0.545097,1.635348,0.773102
189,-0.980576,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.381727,1.057841,-0.752485,0.320513,0.521971,1.689944,0.738739
190,-0.987503,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.359803,1.045153,-0.744649,0.293608,0.498846,1.737741,0.704377


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      174.261
1      164.909
2      202.662
3      174.149
4      159.955
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Rio Grande Do Sul - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
0,1.274013,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.577787,-1.677139,-1.973850,-2.280022,-2.069914,-1.362004,-1.691101
1,1.259068,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.629575,-1.660252,-1.934055,-2.253080,-2.045065,-1.371141,-1.675868
2,1.244123,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.640416,-1.643365,-1.894260,-2.226138,-2.020217,-1.382062,-1.660635
3,1.229178,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.692102,-1.626477,-1.854466,-2.199196,-1.995368,-1.392283,-1.645402
4,1.214233,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.746144,-1.609590,-1.814671,-2.172254,-1.970520,-1.404146,-1.630170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.092018,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.647118,1.197586,-0.541459,0.739492,0.925913,0.936850,1.430588
158,-0.148891,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.689363,1.198866,-0.569240,0.731546,0.918346,0.935032,1.419848
159,-0.205765,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.746341,1.200147,-0.597022,0.723600,0.910778,0.935266,1.409108
160,-0.262638,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.811207,1.201428,-0.624803,0.715654,0.903211,0.937596,1.398368


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      174.261
1      164.909
2      202.662
3      174.149
4      159.955
        ...   
157    196.675
158    251.395
159    199.081
160    220.793
161    211.546
Name: Rio Grande Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
126,-1.124877,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.966618,0.843865,0.507575,0.979193,1.048843,0.602074,0.845653
127,-1.048798,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.910075,0.861669,0.468267,0.975275,1.050210,0.631855,0.868562
128,-0.972719,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.848166,0.879474,0.428960,0.971356,1.051577,0.662142,0.891471
129,-0.896640,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.792181,0.897278,0.389653,0.967437,1.052943,0.695357,0.914380
130,-0.820561,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.728102,0.915082,0.350345,0.963518,1.054310,0.719249,0.937288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.966721,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.420132,1.083219,-0.768157,0.374323,0.568222,1.591021,0.807464
188,-0.973649,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.417824,1.070530,-0.760321,0.347418,0.545097,1.635348,0.773102
189,-0.980576,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.381727,1.057841,-0.752485,0.320513,0.521971,1.689944,0.738739
190,-0.987503,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.359803,1.045153,-0.744649,0.293608,0.498846,1.737741,0.704377


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3146620416, 2361369165, 1756459326, 2380349961, 3998463796, 2053199903, 4072425221, 224084051, 133528371, 1755337964, 2905371763, 1492516551, 1926325253, 1147210775, 1414058152, 1264589988, 74529435, 1734885071, 3035852409, 2929259077, 442612151, 3444089871, 2237146453, 2732274928, 2215661469, 2854131284, 1796658911, 2504167519, 4239166110, 2121046830, 1162002205, 1784788773, 4046595821, 66263290, 2006337982, 2801475284, 387839572, 2395298387, 2613391760, 3355707511, 4203721643, 3458473730, 1404525407, 31312390, 1730066411, 3307664794, 118443433, 4176595203, 3939089774, 4011137541]


Step: 0 ___________________________________________
val_loss: 96.7274169921875
winner_seed: 3146620416


Step: 1 ___________________________________________
val_loss: 88.76972961425781
winner_seed: 2361369165


Step: 2 ___________________________________________
val_loss: 74.78661346435547
winner_seed: 1756459326


Step: 3 ___________________________________________
val_loss: 129.64572143554688


Step: 4 

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 221ms/step - loss: 45376.2227 - val_loss: 115960.8047
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 306113.3750 - val_loss: 461088.6562
Epoch 3/10000
4/4 [==============================] - 0s 12ms/step - loss: 166913.9219 - val_loss: 1718.7034
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 1596.9124 - val_loss: 1104.8965
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 1426.6304 - val_loss: 1218.9651
Epoch 6/10000
4/4 [==============================] - 0s 13ms/step - loss: 1758.4198 - val_loss: 1038.7455
Epoch 7/10000
4/4 [==============================] - 0s 13ms/step - loss: 1377.6936 - val_loss: 937.4218
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 1289.0409 - val_loss: 12238.8281
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 5886.3496 - val_loss: 758.0322
Epoch 10/10000
4/4 [=================

Epoch 79/10000
4/4 [==============================] - 0s 12ms/step - loss: 278.8444 - val_loss: 117.5554
Epoch 80/10000
4/4 [==============================] - 0s 12ms/step - loss: 277.8666 - val_loss: 375.7179
Epoch 81/10000
4/4 [==============================] - 0s 13ms/step - loss: 293.8856 - val_loss: 127.3540
Epoch 82/10000
4/4 [==============================] - 0s 14ms/step - loss: 251.3518 - val_loss: 159.6588
Epoch 83/10000
4/4 [==============================] - 0s 13ms/step - loss: 267.2231 - val_loss: 123.1180
Epoch 84/10000
4/4 [==============================] - 0s 13ms/step - loss: 249.4094 - val_loss: 154.8089
Epoch 85/10000
4/4 [==============================] - 0s 12ms/step - loss: 260.6962 - val_loss: 142.8937
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 249.8219 - val_loss: 144.5234
Epoch 87/10000
4/4 [==============================] - 0s 13ms/step - loss: 268.6886 - val_loss: 134.2548
Epoch 88/10000
4/4 [==============================] - 0

4/4 [==============================] - 0s 14ms/step - loss: 258.5404 - val_loss: 165.5809
Epoch 157/10000
4/4 [==============================] - 0s 15ms/step - loss: 269.8087 - val_loss: 159.1767
Epoch 158/10000
4/4 [==============================] - 0s 14ms/step - loss: 264.2741 - val_loss: 110.7261
Epoch 159/10000
4/4 [==============================] - 0s 14ms/step - loss: 243.1092 - val_loss: 110.6055
Epoch 160/10000
4/4 [==============================] - 0s 14ms/step - loss: 237.6404 - val_loss: 103.8992
Epoch 161/10000
4/4 [==============================] - 0s 14ms/step - loss: 237.9131 - val_loss: 121.5609
Epoch 162/10000
4/4 [==============================] - 0s 14ms/step - loss: 250.8126 - val_loss: 132.3080
Epoch 163/10000
4/4 [==============================] - 0s 15ms/step - loss: 259.1858 - val_loss: 107.0586
Epoch 164/10000
4/4 [==============================] - 0s 14ms/step - loss: 265.3127 - val_loss: 139.2975
Epoch 165/10000
4/4 [==============================] - 0s 14ms

4/4 [==============================] - 0s 15ms/step - loss: 237.9556 - val_loss: 100.1455
Epoch 234/10000
4/4 [==============================] - 0s 15ms/step - loss: 256.8872 - val_loss: 98.7475
Epoch 235/10000
4/4 [==============================] - 0s 15ms/step - loss: 246.4066 - val_loss: 158.0455
Epoch 236/10000
4/4 [==============================] - 0s 16ms/step - loss: 257.5825 - val_loss: 112.4900
Epoch 237/10000
4/4 [==============================] - 0s 18ms/step - loss: 236.8066 - val_loss: 102.8343
Epoch 238/10000
4/4 [==============================] - 0s 16ms/step - loss: 225.7667 - val_loss: 114.0536
Epoch 239/10000
4/4 [==============================] - 0s 15ms/step - loss: 247.9715 - val_loss: 121.0173
Epoch 240/10000
4/4 [==============================] - 0s 14ms/step - loss: 240.7391 - val_loss: 97.3334
Epoch 241/10000
4/4 [==============================] - 0s 12ms/step - loss: 221.0086 - val_loss: 109.3148
Epoch 242/10000
4/4 [==============================] - 0s 13ms/s

4/4 [==============================] - 0s 12ms/step - loss: 231.5098 - val_loss: 119.0562
Epoch 311/10000
4/4 [==============================] - 0s 13ms/step - loss: 243.8899 - val_loss: 118.6575
Epoch 312/10000
4/4 [==============================] - 0s 12ms/step - loss: 221.1164 - val_loss: 113.8526
Epoch 313/10000
4/4 [==============================] - 0s 11ms/step - loss: 234.9440 - val_loss: 119.7448
Epoch 314/10000
4/4 [==============================] - 0s 12ms/step - loss: 246.4420 - val_loss: 113.4701
Epoch 315/10000
4/4 [==============================] - 0s 12ms/step - loss: 246.8640 - val_loss: 101.9907
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 247.4848 - val_loss: 107.7032
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 238.1130 - val_loss: 130.5020
Epoch 318/10000
4/4 [==============================] - 0s 12ms/step - loss: 223.1707 - val_loss: 131.1687
Epoch 319/10000
4/4 [==============================] - 0s 12ms

...

4/4 [==============================] - 0s 10ms/step - loss: 191.9814 - val_loss: 119.7131
Epoch 773/10000
4/4 [==============================] - 0s 10ms/step - loss: 200.6418 - val_loss: 106.8993
Epoch 774/10000
4/4 [==============================] - 0s 11ms/step - loss: 205.1841 - val_loss: 110.2102
Epoch 775/10000
4/4 [==============================] - 0s 10ms/step - loss: 225.2892 - val_loss: 174.2032
Epoch 776/10000
4/4 [==============================] - 0s 11ms/step - loss: 197.5411 - val_loss: 104.3624
Epoch 777/10000
4/4 [==============================] - 0s 10ms/step - loss: 195.0293 - val_loss: 145.6864
Epoch 778/10000
4/4 [==============================] - 0s 11ms/step - loss: 247.7227 - val_loss: 150.0273
Epoch 779/10000
4/4 [==============================] - 0s 10ms/step - loss: 219.5063 - val_loss: 160.7849
Epoch 780/10000
4/4 [==============================] - 0s 10ms/step - loss: 213.3087 - val_loss: 158.0421
Epoch 781/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 10ms/step - loss: 181.4797 - val_loss: 118.2902
Epoch 850/10000
4/4 [==============================] - 0s 10ms/step - loss: 186.0842 - val_loss: 122.0107
Epoch 851/10000
4/4 [==============================] - 0s 10ms/step - loss: 174.9376 - val_loss: 143.4002
Epoch 852/10000
4/4 [==============================] - 0s 10ms/step - loss: 183.2369 - val_loss: 137.3189
Epoch 853/10000
4/4 [==============================] - 0s 10ms/step - loss: 173.2163 - val_loss: 122.6122
Epoch 854/10000
4/4 [==============================] - 0s 10ms/step - loss: 191.5582 - val_loss: 163.9492
Epoch 855/10000
4/4 [==============================] - 0s 10ms/step - loss: 180.6578 - val_loss: 132.5403
Epoch 856/10000
4/4 [==============================] - 0s 10ms/step - loss: 179.5850 - val_loss: 159.3957
Epoch 857/10000
4/4 [==============================] - 0s 11ms/step - loss: 199.9447 - val_loss: 157.4422
Epoch 858/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 10ms/step - loss: 194.9031 - val_loss: 133.2012
Epoch 927/10000
4/4 [==============================] - 0s 10ms/step - loss: 247.8688 - val_loss: 180.0282
Epoch 928/10000
4/4 [==============================] - 0s 10ms/step - loss: 205.7728 - val_loss: 135.4202
Epoch 929/10000
4/4 [==============================] - 0s 10ms/step - loss: 192.3515 - val_loss: 133.7075
Epoch 930/10000
4/4 [==============================] - 0s 10ms/step - loss: 212.5064 - val_loss: 144.6025
Epoch 931/10000
4/4 [==============================] - 0s 10ms/step - loss: 223.5590 - val_loss: 167.4178
Epoch 932/10000
4/4 [==============================] - 0s 10ms/step - loss: 203.0703 - val_loss: 180.1634
Epoch 933/10000
4/4 [==============================] - 0s 10ms/step - loss: 206.0887 - val_loss: 183.8053
Epoch 934/10000
4/4 [==============================] - 0s 10ms/step - loss: 198.3658 - val_loss: 144.9102
Epoch 935/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 11ms/step - loss: 186.3438 - val_loss: 152.4934
Epoch 1004/10000
4/4 [==============================] - 0s 10ms/step - loss: 179.3131 - val_loss: 148.8552
Epoch 1005/10000
4/4 [==============================] - 0s 10ms/step - loss: 188.5933 - val_loss: 195.4028
Epoch 1006/10000
4/4 [==============================] - 0s 10ms/step - loss: 199.3951 - val_loss: 133.4197
Epoch 1007/10000
4/4 [==============================] - 0s 11ms/step - loss: 177.6217 - val_loss: 138.6228
Epoch 1008/10000
4/4 [==============================] - 0s 11ms/step - loss: 173.9086 - val_loss: 164.0406
Epoch 1009/10000
4/4 [==============================] - 0s 10ms/step - loss: 196.8797 - val_loss: 136.0933
Epoch 1010/10000
4/4 [==============================] - 0s 10ms/step - loss: 179.8686 - val_loss: 156.2188
Epoch 1011/10000
4/4 [==============================] - 0s 10ms/step - loss: 177.2819 - val_loss: 143.0377
Epoch 1012/10000
4/4 [==============================] 

Epoch 1080/10000
4/4 [==============================] - 0s 11ms/step - loss: 205.5601 - val_loss: 207.2190
Epoch 1081/10000
4/4 [==============================] - 0s 10ms/step - loss: 193.4842 - val_loss: 159.3757
Epoch 1082/10000
4/4 [==============================] - 0s 10ms/step - loss: 180.2824 - val_loss: 156.9985
Epoch 1083/10000
4/4 [==============================] - 0s 10ms/step - loss: 151.7375 - val_loss: 151.5699
Epoch 1084/10000
4/4 [==============================] - 0s 10ms/step - loss: 157.9745 - val_loss: 165.2727
Epoch 1085/10000
4/4 [==============================] - 0s 10ms/step - loss: 178.8849 - val_loss: 211.0699
Epoch 1086/10000
4/4 [==============================] - 0s 10ms/step - loss: 199.3601 - val_loss: 273.7654
Epoch 1087/10000
4/4 [==============================] - 0s 10ms/step - loss: 264.2390 - val_loss: 151.4613
Epoch 1088/10000
4/4 [==============================] - 0s 10ms/step - loss: 202.5238 - val_loss: 161.2505
Epoch 1089/10000
4/4 [===============

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,217.039597,216.914902,216.717209,216.638947,216.417526,216.248718,215.98587,212.577499,210.353531,210.222336,210.16832,210.009689,209.374237,209.01683,208.71582,207.995834,206.772629,204.304916,201.560425,201.649719,201.413315,201.673599,202.347733,201.72113,201.321411,200.550018,200.285034,199.741074,199.640884,198.813232
Target,244.68,252.183,249.403,208.299,238.414,213.179,221.534,212.139,228.994,226.827,186.855,253.021,194.988,244.406,201.372,251.369,242.782,201.288,226.547,228.5,223.896,234.132,207.586,228.583,234.565,249.519,230.086,233.136,233.8,223.214
Error,27.640396,35.268097,32.685791,8.339951,21.996475,3.069717,5.548126,0.438492,18.640472,16.60466,23.313324,43.011307,14.38623,35.389175,7.343826,43.373169,36.009369,3.016922,24.986572,26.850281,22.482681,32.458405,5.238266,26.861862,33.243591,48.968979,29.800964,33.394928,34.159119,24.400772


In [38]:
display(mae)
display(mape)

23.964064

0.102456905

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[2569.294]] - Target[2735.5280000000002]| =  Error: [[166.23413]]; MAPE:[[0.06076857]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[2456.5461]] - Target[2685.449]| =  Error: [[228.90283]]; MAPE:[[0.0852382]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[1200.3517]] - Target[1404.32]| =  Error: [[203.96826]]; MAPE:[[0.14524344]]


[array([[166.23413]], dtype=float32),
 array([[228.90283]], dtype=float32),
 array([[203.96826]], dtype=float32)]

199.70174

0.0970834